# Project #1 - Exploratory data analysis

## Import used packages

In [30]:
import pandas as pd # dataframes
import numpy as np # matrices and linear algebra
import matplotlib.pyplot as plt # plotting
import seaborn as sns # another matplotlib interface - styled and easier to use

## Loading data from - in our case it is a csv file

In [31]:
df_full = pd.read_csv('./googleplaystore.csv', sep=',')

## Let's take a look at the basic dataset information

### Dataset shape?
- 13 cols, 10841 rows


### Dataset cols names with datatypes?

In [32]:
df_full.dtypes

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

### Are there any missing values?
- Mostly not, a few columns have negligibly few missing values
- Rating has cca 1/11 of records empty

In [33]:
df_full.isna().sum().sort_values()

App                  0
Category             0
Reviews              0
Size                 0
Installs             0
Price                0
Genres               0
Last Updated         0
Type                 1
Content Rating       1
Android Ver          3
Current Ver          8
Rating            1474
dtype: int64

### Removing records without value
- Since the dataset is large enough and the data are not dependent on each other, I choose the option that deletes records without value

In [34]:
df_full = df_full.dropna()

## Now let's write down a few unknowns we wanna find out
1. sdafasdfasf
2. sdafasdfasf
